In [34]:
"""
    post_to_slack.py

When this script is run, it will check the difference in crypto wallet balances between two
# dates and if the difference in balance is significant, it will send a notification to the
# FUND3 slack channel
"""

import datetime
import sys
import json
import blocksci
import numpy as np
import pandas as pd

# sys.path.append("..")
sys.path.append("../../")

from slackclient import SlackClient
from services.transactions import get_top_transactions
pd.set_option('display.max_colwidth', -1)



class failedPost(Exception):
    pass

"""
Simple Util class to send a slack message notification.
"""
class SlackUtil:
    """
    Official Slack API Reference: https://github.com/slackapi/python-slackclient
    Simple Util class to send a slack message notification.
    For now, the functionality is rather simple - just sending a message.
    But in the future logging should be incorporated and this could be developed
    into a full slackbot wrapper.
    TODOs:
    1. Figure out how to search and store all slack channel/ user IDs and
    keep a list instead of using #channelname
    """
    def __init__(self, auth_token):
        """
        Need a better way to store auth_tokens aside from env vars.
        :param auth_token:
        """
        self.client = SlackClient(auth_token)

    def send_message(self, channel, message, botname):
        """
        Sends a message to a channel with botname.
        TODO (calvinleungyk): Extend to adding icon/ emoji url for the bot.
        Example usage:
            slack_util = SlackUtil(auth_token)
            slack_util.send_message('random', 'This is a test message.',
                                    'botname')
        :param channel: str, name of the channel to send to, the
        channel needs to already exist for the call to be successful.
        :param message: str, the actual message.
        :param botname: str, name of the bot that sends the message.
        :return:
        """
        channel = '#'+channel

        # is response an unused variable name ?
        response = self.client.api_call(
            "chat.postMessage",
            channel=channel,
            text=message,
            as_user=False,
            username=botname
        )



# -----------  RETRIEVING TRANSACTION INFORMATION ----------------

VALUE_THRESHOLD = 1500

# Importing authentication token
with open("../../services/token.json") as datafile:
    data = json.load(datafile)
auth_token = data['token']

# Initialising the class
slack_util = SlackUtil(auth_token)

# retrieving top transactions from the last week

d = datetime.date.today()

currentObj = d - datetime.timedelta(days=0)
latest_date = currentObj.strftime('%Y-%-m-%d')

weekAgoObj = d - datetime.timedelta(days=7)
week_ago = weekAgoObj.strftime('%Y-%-m-%d')

s = get_top_transactions(week_ago, latest_date)

# Finding the highest value transaction and its corresponding address
max_value = s['Value'].max()
max_address = s[s['Value'] == s['Value'].max()]['Address']

address_str = str(max_address).split()[1][:-1]+')'

max_date = s[s['Value'] == s['Value'].max()]['Date']
date_str = str(max_date)[6:30]

In [35]:
latest_date

'2018-10-23'

In [36]:
week_ago

'2018-10-16'

In [37]:
max_value

nan

In [44]:
s = get_top_transactions('2018-10-01', '2018-10-23')
s

,Value,Date,Address
0,1521.998769,Wed Oct 10 23:28:22 2018,"ScriptHashAddress(3LvLL5foUDSW1UoUqEfBsFAvqdDTEyDPQ4, wrapped_address=bc1qm3k04txp97m3057wsh2m87e6hz092hy8e9t9a9)"


In [39]:
max_value = s['Value'].max()
max_value

1521.9987693590001

In [40]:
isinstance(max_value, float)

True

In [41]:
address = s[s['Value'] == s['Value'].max()]['Address']

In [42]:
str(address)

'0    ScriptHashAddress(3LvLL5foUDSW1UoUqEfBsFAvqdDTEyDPQ4, wrapped_address=bc1qm3k04txp97m3057wsh2m87e6hz092hy8e9t9a9)\nName: Address, dtype: object'

In [43]:
address_str = str(address).split()[1][:-1]+')'

print(str(address_str))

ScriptHashAddress(3LvLL5foUDSW1UoUqEfBsFAvqdDTEyDPQ4)
